In [1]:
import csv
import os
from fractions import Fraction

import numpy as np
import pandas as pd
from IPython.display import display, Markdown, HTML

pd.set_option('display.precision', 3)
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Border, Side

# Функции

In [2]:
def parse_value(value):
    value = value.strip()
    if not value:
        return None
    if '/' in value:
        numerator, denominator = value.split('/')
        return Fraction(int(numerator), int(denominator))
    else:
        return int(value)

In [3]:
def format_fraction(x):
    return f"{x.real}" if isinstance(x, Fraction) else str(x)

In [4]:
def get_matrix_paired_comparisons(data, names, title):
    df = pd.DataFrame(data, index=names, columns=names)
    styled_df = df.map(format_fraction)
    display(Markdown(f"#### {title}"))
    display(styled_df)
    
    return df

In [5]:
def get_normalized_matrix_paired_comparisons(data, names, title):
    matrix = np.array([[float(item) for item in row] for row in data])

    column_sums = matrix.sum(axis=0)
    
    local_normalized_matrix = matrix / column_sums
    
    df = pd.DataFrame(local_normalized_matrix, index=names, columns=names)
    display(Markdown(f"#### {title}"))
    display(df)
    return local_normalized_matrix

In [6]:
def get_priority_vectors(param_normalized_matrix, names):
    priority_vector = param_normalized_matrix.mean(axis=1)
    return pd.DataFrame(priority_vector, index=names, columns=['Вектор приоритетов'])

In [7]:
def get_lambda_max(matrix, vector):
    np_matrix = np.array(matrix)
    np_vector = np.array(vector)
    matrix_dot_vector = np.dot(np_matrix, np_vector)

    return (1 / len(np_matrix)) * np.sum(matrix_dot_vector / vector, axis=0)

In [8]:
def get_consistency_index(param_lambda_max, quantity):
    return (param_lambda_max - quantity) / (quantity - 1)

In [9]:
def get_random_consistency_index(n):
    random_consistency_indexes =  np.array([0, 0, 0.58, 0.90, 1.12, 1.24, 1.32, 1.41, 1.45, 1.49])
    return random_consistency_indexes[n - 1]

In [10]:
def get_consistency_relation(param_consistency_index, n):
    return param_consistency_index / get_random_consistency_index(n)

In [11]:
def save_to_xlsx(data, val_1, val_2, val_3, name):
    try:
        # Создаем копию DataFrame
        df = data.copy()
        
        # Функция для преобразования значений
        def format_value(x):
            if hasattr(x, 'numerator'):  # Если это дробь
                if x.denominator == 1:  # Если знаменатель = 1
                    return str(x.numerator)  # Выводим только числитель
                return f"{x.numerator}/{x.denominator}"  # Иначе обычная дробь
            elif isinstance(x, (float, int)):  # Если число
                return f"{float(x):.3f}"  # Округляем до 3 знаков
            return str(x)  # Для остальных случаев
        
        # Создаем новую книгу Excel
        wb = Workbook()
        ws = wb.active
        
        # Добавляем пустую угловую ячейку (A1)
        ws.cell(row=1, column=1, value="")
        
        # Записываем заголовки столбцов (начиная с B1)
        for col_idx, col_name in enumerate(df.columns, 2):
            ws.cell(row=1, column=col_idx, value=col_name)
        
        # Записываем данные с названиями строк
        for row_idx, (index, row) in enumerate(df.iterrows(), 2):
            # Название строки в первом столбце
            ws.cell(row=row_idx, column=1, value=str(index))
            # Данные строки
            for col_idx, value in enumerate(row, 2):
                ws.cell(row=row_idx, column=col_idx, value=format_value(value))
        
        # Устанавливаем ширину столбцов
        for col in ws.columns:
            max_length = 0
            column = col[0].column_letter
            
            for cell in col:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(str(cell.value))
                except:
                    pass
            
            adjusted_width = (max_length + 2) * 1.2
            ws.column_dimensions[column].width = adjusted_width
        
        last_col = df.shape[1] + 1  # +1 потому что добавили столбец с названиями строк
        start_row = df.shape[0] + 2  # Исправлено: было +3, стало +2 (убрали лишнюю строку)
        
        # Стиль границы
        thin_border = Border(left=Side(style='thin'), 
                           right=Side(style='thin'), 
                           top=Side(style='thin'), 
                           bottom=Side(style='thin'))
        
        # Объединяем все три строки в одну большую ячейку
        ws.merge_cells(start_row=start_row, start_column=1, 
                      end_row=start_row+2, end_column=last_col-1)
        
        # Пустая ячейка в объединенной области
        ws.cell(row=start_row, column=1, value="")
        
        # Добавляем значения с подписями
        ws.cell(row=start_row, column=last_col, value=f"λ_max = {val_1:.3f}")
        ws.cell(row=start_row+1, column=last_col, value=f"ИС = {val_2:.3f}")
        ws.cell(row=start_row+2, column=last_col, value=f"ОС = {val_3:.3f}")
        
        # Применяем границы ко всем ячейкам
        for row in ws.iter_rows(min_row=1, max_row=start_row+2, max_col=last_col):
            for cell in row:
                cell.border = thin_border
        
        # Создаем папку для сохранения
        os.makedirs("out", exist_ok=True)
        
        # Сохраняем файл
        file_path = os.path.join("out", f"{name}.xlsx")
        wb.save(file_path)
        
        display(Markdown(f"Файл успешно сохранён: {file_path}"))
        return True
    
    except Exception as e:
        display(Markdown(f"Ошибка при сохранении файла: {e}"))
        return False

# Задание матриц

In [12]:
criteria = []
names_criteria = []
number_alternatives = None

with open('resources/criteria.csv', 'r', encoding='utf-8-sig') as csvfile: 
    csvreader = csv.reader(csvfile, delimiter=';')
    names_criteria = next(csvreader)
    number_alternatives = len(names_criteria)
    for row in csvreader:
        parsed_row = [parse_value(item) for item in row]
        criteria.append(parsed_row)
        
all_alternatives = []
names_alternatives = []
for i in range(number_alternatives):
    all_alternatives.append([])


for alt in range(number_alternatives):
    with open(f'./resources/alternative_{alt + 1}.csv', 'r', encoding='utf-8-sig') as csvfile:
        csvreader = csv.reader(csvfile, delimiter=';')
        names_alternatives = next(csvreader)
        for row in csvreader:
            parsed_row = [parse_value(item) for item in row]
            all_alternatives[alt].append(parsed_row)
            

# Расчёты

## Критерии

In [13]:
matrix_paired_comparisons_alternative = get_matrix_paired_comparisons(criteria, names_criteria, 'Матрица парных сравнений для второго уровня доминантной иерархии')
normalized_matrix_paired_comparisons_alternative = get_normalized_matrix_paired_comparisons(criteria, names_criteria, 'Нормализованная матрица парных сравнений для второго уровня доминантной иерархии')
priority_vectors_alternative = get_priority_vectors(normalized_matrix_paired_comparisons_alternative, names_criteria)

display(Markdown(f"#### Матрица парных сравнений для второго уровня доминантной иерархии с векторами приоритетов"))
result_matrix_criteria = pd.concat([matrix_paired_comparisons_alternative, priority_vectors_alternative], axis=1)
display(result_matrix_criteria)

lambda_max_alternative = get_lambda_max(matrix_paired_comparisons_alternative, priority_vectors_alternative).iloc[0]
consistency_index_criteria = get_consistency_index(lambda_max_alternative, len(criteria))
consistency_relation_criteria =  get_consistency_relation(consistency_index_criteria, len(criteria))

display(Markdown(rf"#### Максимальное собственное значение: $\lambda_{{max}} = {lambda_max_alternative:.4f}$"))
display(Markdown(f"#### Индекс согласованности: ИС = {consistency_index_criteria:.4f}"))
display(Markdown(f"#### Отношение согласованности: ОС = {consistency_relation_criteria:.4f}"))
save_to_xlsx(result_matrix_criteria, lambda_max_alternative, consistency_index_criteria, consistency_relation_criteria, "Матрица_Критериев")
display(HTML("<hr>"))

#### Матрица парных сравнений для второго уровня доминантной иерархии

,Стоимость ТС,Стоимость обслуживания ТС,Надёжность,Безопасность,Дизайн,Комфорт,Мощность,Год выпуска
Стоимость ТС,1,3,2,4,6,5,7,8
Стоимость обслуживания ТС,1/3,1,1/2,2,4,3,5,6
Надёжность,1/2,2,1,3,5,4,6,7
Безопасность,1/4,1/2,1/3,1,3,2,4,5
Дизайн,1/6,1/4,1/5,1/3,1,1/2,2,3
Комфорт,1/5,1/3,1/4,1/2,2,1,3,4
Мощность,1/7,1/5,1/6,1/4,1/2,1/3,1,2
Год выпуска,1/8,1/6,1/7,1/5,1/3,1/4,1/2,1


#### Нормализованная матрица парных сравнений для второго уровня доминантной иерархии

,Стоимость ТС,Стоимость обслуживания ТС,Надёжность,Безопасность,Дизайн,Комфорт,Мощность,Год выпуска
Стоимость ТС,0.368,0.403,0.435,0.355,0.275,0.311,0.246,0.222
Стоимость обслуживания ТС,0.123,0.134,0.109,0.177,0.183,0.187,0.175,0.167
Надёжность,0.184,0.268,0.218,0.266,0.229,0.249,0.211,0.194
Безопасность,0.092,0.067,0.073,0.089,0.137,0.124,0.140,0.139
Дизайн,0.061,0.034,0.044,0.030,0.046,0.031,0.070,0.083
Комфорт,0.074,0.045,0.054,0.044,0.092,0.062,0.105,0.111
Мощность,0.053,0.027,0.036,0.022,0.023,0.021,0.035,0.056
Год выпуска,0.046,0.022,0.031,0.018,0.015,0.016,0.018,0.028


#### Матрица парных сравнений для второго уровня доминантной иерархии с векторами приоритетов

,Стоимость ТС,Стоимость обслуживания ТС,Надёжность,Безопасность,Дизайн,Комфорт,Мощность,Год выпуска,Вектор приоритетов
Стоимость ТС,1,3,2,4,6,5,7,8,0.327
Стоимость обслуживания ТС,1/3,1,1/2,2,4,3,5,6,0.157
Надёжность,1/2,2,1,3,5,4,6,7,0.227
Безопасность,1/4,1/2,1/3,1,3,2,4,5,0.108
Дизайн,1/6,1/4,1/5,1/3,1,1/2,2,3,0.050
Комфорт,1/5,1/3,1/4,1/2,2,1,3,4,0.073
Мощность,1/7,1/5,1/6,1/4,1/2,1/3,1,2,0.034
Год выпуска,1/8,1/6,1/7,1/5,1/3,1/4,1/2,1,0.024


#### Максимальное собственное значение: $\lambda_{max} = 8.2919$

#### Индекс согласованности: ИС = 0.0417

#### Отношение согласованности: ОС = 0.0296

Файл успешно сохранён: out\Матрица_Критериев.xlsx

## Альтернативы

In [14]:
for i in range(number_alternatives):

    alternative = all_alternatives[i]
    display(Markdown(f"### Критерий: {names_criteria[i]}"))

    matrix_paired_comparisons_alternative = get_matrix_paired_comparisons(alternative, names_alternatives, f'Матрица парных сравнений для третьего уровня доминантной иерархии для {names_criteria[i]}')
    normalized_matrix_paired_comparisons_alternative = get_normalized_matrix_paired_comparisons(alternative, names_alternatives, 'Нормализованная матрица парных сравнений для третьего уровня доминантной иерархии')
    priority_vectors_alternative = get_priority_vectors(normalized_matrix_paired_comparisons_alternative, names_alternatives)

    display(Markdown(f"#### Матрица парных сравнений для третьего уровня доминантной иерархии для {names_criteria[i]} с векторами приоритетов"))
    display(pd.concat([matrix_paired_comparisons_alternative, priority_vectors_alternative], axis=1))

    lambda_max_alternative = get_lambda_max(matrix_paired_comparisons_alternative, priority_vectors_alternative).iloc[0]
    consistency_index_alternative = get_consistency_index(lambda_max_alternative, len(alternative))
    consistency_relation_alternative =  get_consistency_relation(consistency_index_alternative, len(alternative))

    display(Markdown(rf"#### Максимальное собственное значение: $\lambda_{{max}} = {lambda_max_alternative:.4f}$"))
    display(Markdown(f"#### Индекс согласованности: ИС = {consistency_index_alternative:.4f}"))
    display(Markdown(f"#### Отношение согласованности: ОС = {consistency_relation_alternative:.4f}"))
    save_to_xlsx(result_matrix_criteria, lambda_max_alternative, consistency_index_criteria, consistency_relation_criteria, f"Матрица_Альтернатив_{names_criteria[i]}")
    display(HTML("<hr>"))

### Критерий: Стоимость ТС

#### Матрица парных сравнений для третьего уровня доминантной иерархии для Стоимость ТС

,Kia Rio,Volkswagen Golf,Toyota Corolla,Skoda Octavia,BMW 3 Series,Hyundai Solaris
Kia Rio,1,3,2,2,5,1
Volkswagen Golf,1/3,1,1/2,1/2,3,1/3
Toyota Corolla,1/2,2,1,1,4,1/2
Skoda Octavia,1/2,2,1,1,4,1/2
BMW 3 Series,1/5,1/3,1/4,1/4,1,1/5
Hyundai Solaris,1,3,2,2,5,1


#### Нормализованная матрица парных сравнений для третьего уровня доминантной иерархии

,Kia Rio,Volkswagen Golf,Toyota Corolla,Skoda Octavia,BMW 3 Series,Hyundai Solaris
Kia Rio,0.283,0.265,0.296,0.296,0.227,0.283
Volkswagen Golf,0.094,0.088,0.074,0.074,0.136,0.094
Toyota Corolla,0.142,0.176,0.148,0.148,0.182,0.142
Skoda Octavia,0.142,0.176,0.148,0.148,0.182,0.142
BMW 3 Series,0.057,0.029,0.037,0.037,0.045,0.057
Hyundai Solaris,0.283,0.265,0.296,0.296,0.227,0.283


...

,Kia Rio,Volkswagen Golf,Toyota Corolla,Skoda Octavia,BMW 3 Series,Hyundai Solaris,Вектор приоритетов
Kia Rio,1,3,1/2,1,5,3,0.221
Volkswagen Golf,1/3,1,1/3,1/3,3,1,0.093
Toyota Corolla,2,3,1,2,7,3,0.333
Skoda Octavia,1,3,1/2,1,5,3,0.221
BMW 3 Series,1/5,1/3,1/7,1/5,1,1/3,0.039
Hyundai Solaris,1/3,1,1/3,1/3,3,1,0.093


#### Максимальное собственное значение: $\lambda_{max} = 6.1031$

#### Индекс согласованности: ИС = 0.0206

#### Отношение согласованности: ОС = 0.0166

Файл успешно сохранён: out\Матрица_Альтернатив_Год выпуска.xlsx